# Pre-Model Training Foundations and Imports

In [39]:
# Import Dependencies
import numpy as np
import pandas as pd
import os

In [40]:
# Mounting Google Drive and setting path to import data
from google.colab import drive
drive.mount("/content/drive")
path = "drive/MyDrive/HUDK4050 Midterm Project/notebooks/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# Import and create DataFrame for kaggletest student IDs
kaggletest_df = pd.read_csv(path + "testData.csv")

kaggletest_df

,StudentID
0,344883
1,298782
2,311983
3,346945
4,313391
...,...
995,310102
996,329407
997,323071
998,356228


# Pre-processing and Feature Engineering Kaggletest Data

In [42]:
# Define a new function to load all CSV files from within a folder
def load_csv_files_from_folder(path):
  files = os.listdir(path)
  # Create a list of DataFrames from all files with .csv extension
  df_list = [pd.read_csv(os.path.join(path, file)) for file in files if file.endswith(".csv")]
  # Return concatenated list from the list of DataFrames
  return pd.concat(df_list, ignore_index = True)

In [43]:
# Import student static data DataFrames
static_df = load_csv_files_from_folder(path + "Student Static Data")

# Import financial aid DataFrame
financial_df = pd.read_excel(path + "fin_aid_fasfa_data.xlsx")
# Replace student ID column index to standardize StudentID
financial_df.rename(columns = {"ID with leading" : "StudentID"}, inplace = True)

# Import all student progress DataFrames
progress_df = load_csv_files_from_folder(path + "Student Progress Data")

In [44]:
# Merging student financial aid, static data, and progress DataFrames into one
merged_df = kaggletest_df.merge(static_df, on = "StudentID", how = "left")
merged_df = merged_df.merge(financial_df, on = "StudentID", how = "left")
merged_df = merged_df.merge(progress_df, on = "StudentID", how = "left")

# Display the final merged DataFrame
merged_df

,StudentID,Cohort_x,CohortTerm_x,Campus,Address1,Address2,City,State,Zip,RegistrationDate,...,Major1,Major2,Complete1,Complete2,CompleteCIP1,CompleteCIP2,TransferIntent,DegreeTypeSought,TermGPA,CumGPA
0,344883,2015-16,1,NaN,102 Vermont Ave,NaN,Newark,NJ,7106.0,20150611,...,52.0201,-1.0,0,0,-2.0000,-2,-1,6,2.38,1.86
1,344883,2015-16,1,NaN,102 Vermont Ave,NaN,Newark,NJ,7106.0,20150611,...,52.0201,-1.0,0,0,-2.0000,-2,-1,6,0.00,0.99
2,344883,2015-16,1,NaN,102 Vermont Ave,NaN,Newark,NJ,7106.0,20150611,...,52.0201,-1.0,0,0,-2.0000,-2,-1,6,1.00,1.00
3,298782,2011-12,1,NaN,405 Woodlawn Ave,NaN,Jersey City,NJ,7305.0,20110516,...,0.0000,-1.0,0,0,-2.0000,-2,-1,6,3.43,3.35
4,298782,2011-12,1,NaN,405 Woodlawn Ave,NaN,Jersey City,NJ,7305.0,20110516,...,0.0000,-1.0,0,0,-2.0000,-2,-1,6,2.15,2.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,356228,2016-17,1,NaN,189 Winfield Ave,NaN,Jersey City,NJ,7305.0,20160629,...,51.3801,-1.0,0,0,-2.0000,-2,-1,6,2.54,2.57
4372,356228,2016-17,1,NaN,189 Winfield Ave,NaN,Jersey City,NJ,7305.0,20160629,...,51.3801,-1.0,0,0,-2.0000,-2,-1,6,2.60,2.60
4373,320283,2013-14,1,NaN,357-363 Lincoln Ave Apt 1D,NaN,Orange,NJ,7050.0,20130416,...,51.3801,-1.0,8,0,51.3801,-2,-1,6,4.00,3.36
4374,320283,2013-14,1,NaN,357-363 Lincoln Ave Apt 1D,NaN,Orange,NJ,7050.0,20130416,...,51.3801,-1.0,0,0,-2.0000,-2,-1,6,2.94,3.32


# Cleaning the Merged Student Characteristic Dataset

In [45]:
# Cleaning dataset and only keep the last term each student attended
last_term_attended = merged_df.groupby("StudentID").tail(1)

# Ensuring ordering of StudentIDs maintained as in original dataset
cleaned_df = merged_df[merged_df["StudentID"].isin(merged_df["StudentID"])].drop_duplicates("StudentID")

cleaned_df

,StudentID,Cohort_x,CohortTerm_x,Campus,Address1,Address2,City,State,Zip,RegistrationDate,...,Major1,Major2,Complete1,Complete2,CompleteCIP1,CompleteCIP2,TransferIntent,DegreeTypeSought,TermGPA,CumGPA
0,344883,2015-16,1,NaN,102 Vermont Ave,NaN,Newark,NJ,7106.0,20150611,...,52.0201,-1.0,0,0,-2.0000,-2,-1,6,2.38,1.86
3,298782,2011-12,1,NaN,405 Woodlawn Ave,NaN,Jersey City,NJ,7305.0,20110516,...,0.0000,-1.0,0,0,-2.0000,-2,-1,6,3.43,3.35
6,311983,2012-13,1,NaN,509 76th St Apt 1,NaN,North Bergen,NJ,7047.0,20120522,...,11.0101,-1.0,0,0,-2.0000,-2,-1,6,0.00,2.38
18,346945,2015-16,1,NaN,248 Reynolds Ter Apt 1E,NaN,Orange,NJ,7050.0,20150528,...,26.0101,-1.0,0,0,-2.0000,-2,-1,6,1.59,2.58
22,313391,2012-13,1,NaN,61 Cottage St 1st Floor,NaN,Bayonne,NJ,7002.0,20120611,...,23.0101,-1.0,0,0,-2.0000,-2,-1,6,3.70,3.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4346,310102,2012-13,1,NaN,72 Florida Ave Fl 2,NaN,Paterson,NJ,7011.0,20120703,...,45.1001,-1.0,0,0,-2.0000,-2,-1,6,2.00,2.57
4356,329407,2014-15,1,NaN,2601 New York Ave Apt 1,NaN,Union City,NJ,7087.0,20140616,...,43.0199,-1.0,8,0,43.0199,-2,-1,6,3.50,2.77
4364,323071,2013-14,1,NaN,708 29th St Apt 1,NaN,Union City,NJ,7087.0,20130813,...,54.0101,-1.0,0,0,-2.0000,-2,-1,6,0.00,1.86
4371,356228,2016-17,1,NaN,189 Winfield Ave,NaN,Jersey City,NJ,7305.0,20160629,...,51.3801,-1.0,0,0,-2.0000,-2,-1,6,2.54,2.57


In [46]:
# Check for amount of missing values in data
missing = cleaned_df.isnull().sum()
missing_percent = (missing / len(cleaned_df)) * 100

# Create a DataFrame to view missing values and percentage
missing_df = pd.DataFrame({"Missing Values" : missing, "Percentage" : missing_percent})
missing_df_sorted = missing_df[missing_df["Missing Values"] > 0].sort_values(
    by = "Percentage", ascending = False)

missing_df_sorted.head(5)

,Missing Values,Percentage
Campus,1000,100.0
2012 Work/Study,993,99.3
2012 Scholarship,989,98.9
2013 Work/Study,985,98.5
2014 Work/Study,983,98.3


## Fill in Missing Values Within Merged DataFrame

In [47]:
# Set a threshold of 60% for dropping columns with more than these missing values
threshold = 60

# Drop columns with missing values with greater percentage than these
columns_drop = missing_df_sorted[missing_df_sorted["Percentage"] > threshold].index
cleaned_df = cleaned_df.drop(columns = columns_drop)

In [48]:
# Check for missing values in the remaining dataset
remaining_missing = cleaned_df.isnull().sum()
remaining_missing_df = pd.DataFrame({"Remaining Missing Values" : remaining_missing})
remaining_missing_df = remaining_missing_df[remaining_missing_df["Remaining Missing Values"] > 0]

# Display the remaining missing values
remaining_missing_df.head(10)

,Remaining Missing Values
Address1,10
City,10
State,10
Zip,13
Marital Status,151
Adjusted Gross Income,151
Parent Adjusted Gross Income,151
Father's Highest Grade Level,159
Mother's Highest Grade Level,178
Housing,151


In [49]:
# Import additional dependencies
from sklearn.impute import SimpleImputer

# Create a imputer for "most_frequent" for categorical columns
categorical_imputer = SimpleImputer(strategy = "most_frequent")
categorical_columns = ["Marital Status", "Father's Highest Grade Level", "Mother's Highest Grade Level"]

# Create a imputer for "median" for numerical columns
numerical_imputer = SimpleImputer(strategy = "median")
numerical_columns = ["Adjusted Gross Income", "Parent Adjusted Gross Income"]

In [50]:
# Impute the categorical columns
cleaned_df[categorical_columns] = categorical_imputer.fit_transform(cleaned_df[categorical_columns])

# Impute the numerical columns
cleaned_df[numerical_columns] = numerical_imputer.fit_transform(cleaned_df[numerical_columns])

# Remove geographical data as well as BirthYear
columns_remove = ["Address1", "City", "State", "Zip", "BirthYear"]
cleaned_df = cleaned_df.drop(columns = columns_remove)

In [51]:
# Impute the "Housing" column with most frequent value
cleaned_df["Housing"] = categorical_imputer.fit_transform(cleaned_df[["Housing"]])

## Drop Redundant Categorical Variables and Encode Others

In [52]:
# Drop the potentially redundant categorical variables
columns_drop = ["Cohort_x", "Cohort_y", "AcademicYear", "cohort"]
cleaned_df = cleaned_df.drop(columns = columns_drop)

In [53]:
# Encode remaining categorical variables and use one-hot encoding as default method
categorical_cols = ["Marital Status", "Father's Highest Grade Level",
                    "Mother's Highest Grade Level", "Housing"]
cleaned_df = pd.get_dummies(cleaned_df, columns = categorical_cols)

In [54]:
# Survey where data is currently at
print(cleaned_df.shape)
cleaned_df.columns

(1000, 59)


Index(['StudentID', 'CohortTerm_x', 'RegistrationDate', 'Gender', 'BirthMonth',
       'Hispanic', 'AmericanIndian', 'Asian', 'Black', 'NativeHawaiian',
       'White', 'TwoOrMoreRace', 'HSDip', 'HSDipYr', 'HSGPAUnwtd', 'HSGPAWtd',
       'FirstGen', 'DualHSSummerEnroll', 'EnrollmentStatus',
       'NumColCredAttemptTransfer', 'NumColCredAcceptTransfer',
       'CumLoanAtEntry', 'HighDeg', 'MathPlacement', 'EngPlacement',
       'GatewayMathStatus', 'GatewayEnglishStatus', 'cohort term',
       'Adjusted Gross Income', 'Parent Adjusted Gross Income', 'CohortTerm_y',
       'Term', 'CompleteDevMath', 'CompleteDevEnglish', 'Major1', 'Major2',
       'Complete1', 'Complete2', 'CompleteCIP1', 'CompleteCIP2',
       'TransferIntent', 'DegreeTypeSought', 'TermGPA', 'CumGPA',
       'Marital Status_Divorced', 'Marital Status_Married',
       'Marital Status_Separated', 'Marital Status_Single',
       'Father's Highest Grade Level_College',
       'Father's Highest Grade Level_High School',
  

In [55]:
# One-hot encode race variables into one "Race" feature
races = ["Hispanic", "AmericanIndian", "Asian", "Black", "NativeHawaiian", "White", "TwoOrMoreRace"]

# One-hot encoding the race columns into numerical labels
cleaned_df["Race"] = cleaned_df[races].idxmax(axis = 1).replace({
    "Hispanic" : 1, "AmericanIndian" : 2, "Asian" : 3, "Black" : 4,
    "NativeHawaiian" : 5, "White" : 6, "TwoOrMoreRace" : 7
})
# Drop the original Race binary labeled columns
cleaned_df.drop(races, axis = 1, inplace = True)

In [56]:
# Drop additional columns such as birth month, cohort term, registration date, etc.
columns_remove = [
    "CohortTerm_x", "BirthMonth", "FirstGen", "HSGPAUnwtd", "HSGPAWtd", "FirstGen", "DualHSSummerEnroll",
    "CumLoanAtEntry", "cohort term", "CohortTerm_y", "Term", "Complete2", "TransferIntent", "DegreeTypeSought"
]

In [57]:
# Additional Cleaning for Features which Don't Appear to be Missing
columns_impute = ['NumColCredAttemptTransfer', 'NumColCredAcceptTransfer', 'HighDeg']

median_values = {}
for col in columns_impute:
  valid_values = cleaned_df[(cleaned_df[col] != -1) & (cleaned_df[col] != -2)][col]
  median_values[col] = valid_values.median()

# Replacing -1 values with median values in dataset
for col, median_val in median_values.items():
  cleaned_df.loc[cleaned_df[col] == -1, col] = median_val


# Display calculated median values
median_values

{'NumColCredAttemptTransfer': 69.5,
 'NumColCredAcceptTransfer': 65.0,
 'HighDeg': 0.0}

In [58]:
cleaned_df.shape

(1000, 53)

# Export Cleaned Kaggletest DataFrame to Data Folder

In [59]:
cleaned_df.to_csv(path + "cleaned_kaggletest.csv", index = False)